In [1]:
!pip install langchain groq
!pip install langchain-groq groq

In [2]:
import pandas as pd  # To handle Excel data
import os  # For environment variables
from langchain_groq import ChatGroq  # If using Groq API
from langchain.prompts import PromptTemplate  # For defining templates
from langchain.chains import LLMChain  # For chaining model and prompt

# 1. Load Recipe Dataset from Excel
def load_dataset(file_path):
    """
    Load the recipe dataset from the specified file path.
    """
    try:
        data = pd.read_excel(file_path)
        print("\nRecipe dataset loaded successfully.")
        return data
    except FileNotFoundError:
        print(f"\nError: File not found at {file_path}. Please check the file path.")
        return None
    except Exception as e:
        print(f"\nError loading dataset: {str(e)}")
        return None

# 2. Initialize the LLM
def initialize_llm(api_key, model_name="llama-3.3-70b-versatile"):
    """
    Initialize the Groq LLM with the provided API key and model name.
    """
    try:
        os.environ["GROQ_API_KEY"] = api_key
        return ChatGroq(model_name=model_name)
    except Exception as e:
        print(f"Error initializing LLM: {str(e)}")
        return None

# 3. Generate Recipe from Dataset
def generate_recipe_from_dataset(data, recipe_name):
    """
    Retrieve a recipe by its name from the dataset.
    """
    if data is None:
        return "Error: Recipe dataset is not loaded."

    recipe = data[data['Recipe Name'].str.contains(recipe_name, case=False, na=False)]
    if recipe.empty:
        return f"\n\nNo recipes found with the name '{recipe_name}'."

    recipe = recipe.iloc[0]
    return (
        f"\n\n**Recipe Name:** {recipe['Recipe Name']}\n\n"
        f"**Cuisine:** {recipe['Cuisine']}\n"
        f"**Total Time Required:** {recipe['Total Time Required']}\n\n"
        f"**Ingredients:**\n{recipe['Ingredients']}\n\n"
        f"**Instructions:**\n{recipe['Instructions']}\n\n"
        f"**Nutritional Values:**\n{recipe['Nutritional Values']}"
    )

# 4. Generate Recipe Using AI
def generate_recipe_ai(chain, ingredients):
    """
    Generate a recipe using AI based on the provided ingredients.
    """
    try:
        response = chain.run(ingredients)
        return response
    except Exception as e:
        return f"\n\nError generating recipe: {str(e)}"

# 5. Search Recipes Based on Cuisine
def search_recipes_by_cuisine(data, cuisine):
    """
    Search recipes based on cuisine.
    """
    recipes = data[data['Cuisine'].str.contains(cuisine, case=False, na=False)]
    if recipes.empty:
        return f"\n\nNo recipes found for cuisine '{cuisine}'."
    return recipes[['Recipe Name']].to_string(index=False)

# 6. Search Recipes Based on Ingredient
def search_recipes_by_ingredient(data, ingredient):
    """
    Search recipes based on a specific ingredient.
    """
    recipes = data[data['Ingredients'].str.contains(ingredient, case=False, na=False)]
    if recipes.empty:
        return f"\n\nNo recipes found containing '{ingredient}'."
    return recipes[['Recipe Name']].to_string(index=False)

# 7. Search Nutritional Values for Recipe
def search_nutritional_values(data, chain, recipe_name):
    """
    Search for nutritional values of a recipe. If not found, generate using AI.
    """
    recipe = data[data['Recipe Name'].str.contains(recipe_name, case=False, na=False)]
    if recipe.empty:
        prompt = f"Generate a recipe and nutritional values for '{recipe_name}'."
        response = chain.run(prompt)
        return response
    return recipe.iloc[0]['Nutritional Values']

# 8. Handle User Query
def handle_query(data, chain):
    """
    Handle user queries for recipe search or generation.
    """
    print("\nWelcome to the Recipe Assistant!")
    print("1. Search Recipe by Name")
    print("2. Generate Recipe Using Ingredients")
    print("3. Search Recipes by Cuisine")
    print("4. Search Recipes by Ingredient")
    print("5. Search Nutritional Values for Recipe")
    print("6. Exit")

    query_type = input("\nEnter the query type (1/2/3/4/5/6): ").strip()

    if query_type == "1":
        recipe_name = input("\nEnter the recipe name: ").strip()
        result = generate_recipe_from_dataset(data, recipe_name)
        print(result)

    elif query_type == "2":
        ingredients = input("\nEnter the ingredients (comma-separated): ").strip()
        result = generate_recipe_ai(chain, ingredients)
        print(result)

    elif query_type == "3":
        while True:
            cuisine = input("\nEnter the cuisine type: ").strip()
            result = search_recipes_by_cuisine(data, cuisine)
            print(result)

            if "No recipes found" not in result:
                print("\nOptions:")
                print("1. View a Recipe")
                print("2. Go Back to Main Menu")

                choice = input("Enter your choice (1/2): ").strip()
                if choice == "1":
                    recipe_name = input("\nEnter the recipe name: ").strip()
                    recipe_result = generate_recipe_from_dataset(data, recipe_name)
                    print(recipe_result)
                    break
                elif choice == "2":
                    break
                else:
                    print("\nInvalid input. Please try again.")
            else:
                break

    elif query_type == "4":
        while True:
            ingredient = input("\nEnter the ingredient: ").strip()
            result = search_recipes_by_ingredient(data, ingredient)
            print(result)

            if "No recipes found" not in result:
                print("\nOptions:")
                print("1. View a Recipe")
                print("2. Go Back to Main Menu")

                choice = input("Enter your choice (1/2): ").strip()
                if choice == "1":
                    recipe_name = input("\nEnter the recipe name: ").strip()
                    recipe_result = generate_recipe_from_dataset(data, recipe_name)
                    print(recipe_result)
                    break
                elif choice == "2":
                    break
                else:
                    print("\nInvalid input. Please try again.")
            else:
                break

    elif query_type == "5":
        recipe_name = input("\nEnter the recipe name: ").strip()
        result = search_nutritional_values(data, chain, recipe_name)
        print(result)

    elif query_type == "6":
        print("\nExiting... Goodbye!")
        return False  # Exit the loop

    else:
        print("\nInvalid input. Please try again.")

    return True  # Continue the loop

# 9. Main Function
def main():
    # Define file path and API key
    file_path = "Book1.xlsx"  # Update with your actual file path
    api_key = "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"  # Replace with your API key

    # Load dataset
    data = load_dataset(file_path)
    if data is None:
        return

    # Initialize LLM
    llm = initialize_llm(api_key)
    if llm is None:
        return

    # Create prompt template and chain
    prompt_template = PromptTemplate(
        input_variables=["ingredients"],
        template="\n\nGenerate a recipe using the following ingredients: {ingredients}. Include instructions and estimated total time."
    )
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Interactive query handling
    while True:
        if not handle_query(data, chain):
            break

# Entry point
if __name__ == "__main__":
    main()



Recipe dataset loaded successfully.


C:\Users\krish\AppData\Local\Temp\ipykernel_17280\3037064345.py:208: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)



Welcome to the Recipe Assistant!
1. Search Recipe by Name
2. Generate Recipe Using Ingredients
3. Search Recipes by Cuisine
4. Search Recipes by Ingredient
5. Search Nutritional Values for Recipe
6. Exit

Enter the query type (1/2/3/4/5/6): 6

Exiting... Goodbye!


In [3]:
!pip freeze > requirements.txt